Ingredient1: Rank Data
Ingredient2: Song Name Translation
Ingredient3: metadata



In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from piu_utilities import *



In [32]:
csvFolder = "Ranking_Data/"
start = "202307"
end = "202403"
#Read Data month by month
def iterate_year_month_simple(start, end):
    """Iterate from start to end, where start and end are in 'YYYYMM' format."""
    ans = []

    start_year, start_month = divmod(int(start), 100)
    end_year, end_month = divmod(int(end), 100)
    
    current_year, current_month = start_year, start_month
    
    while (current_year, current_month) <= (end_year, end_month):
        ans.append(f'{current_year}{current_month:02d}')  # Format as 'YYYYMM'
        # Increment month
        current_month += 1
        # If month goes beyond December, increment year and reset month to January
        if current_month > 12:
            current_month = 1
            current_year += 1
    return ans

months = iterate_year_month_simple(start, end)

filename = [f"{csvFolder}{month} ranking.csv" for month in months]

# Read in the data

newest_df = pd.read_csv(filename[len(filename)-1])


In [26]:
#make initial dataframe from songList

Songlist_file = "Metadata-ComposerSongNames.txt"

with open(Songlist_file, encoding = "utf-8") as f:
    songList = f.readlines()

kr_songList = []
kr_composerList = []
en_songList = []
en_composerList = []
for item in songList:
    item = item.split("//")
    kr_songList.append(item[0])
    kr_composerList.append(item[1])
    en_songList.append(item[2])
    en_composerList.append(item[3][:-1])

    
songList_df = pd.DataFrame({'kr_song':kr_songList, 'kr_composer':kr_composerList})
                           # , 'en_song':en_songList, 'en_composer':en_composerList})

songList_df


,kr_song,kr_composer
0,스테이저,Ras
1,리틀 먼치킨,Castellia
2,서든 어페어런스 이미지,Blacklolita
3,톰보이,(여자)아이들
4,"사이먼 세이즈, 유로댄스!! (feat. Sara☆M)",Jehezukiel☆KAZEMURA☆Hideky
...,...,...
539,눈을 감아,반야
540,미운 오리 토카타,반야 프로덕션
541,셋업 미투 믹스,반야 프로덕션
542,유니크 - SHORT CUT -,SHK


In [48]:
#결측값 보정
newest_df['song'] = newest_df['song'].apply(correct_anomalies_custom)

In [50]:
#if there are elements of the songList that are not in the newest_df, print them
print("Songs in songList that are not in the newest_df")
for song in songList_df['kr_song']:
    if song not in newest_df['song'].values:
        print(song)

print(" ")
print("Songs in newest_df that are not in the songList")
songlist = []
for song in newest_df['song']: 
    if song not in songList_df['kr_song'].values and song not in songlist:
        songlist.append(song)
print(songlist)

Songs in songList that are not in the newest_df
팔랑크스 "RS2018 edit"
86 - FULL SONG -
 
Songs in newest_df that are not in the songList
['8 6 - FULL SONG -', '팔랑크스 RS2018 edit""']


In [34]:
import copy
newest_df_copy = copy.deepcopy(newest_df)

newest_df_copy = newest_df_copy.merge(songList_df, left_on = 'song', right_on = 'kr_song', how = 'left')

In [37]:
newest_df_copy = newest_df_copy.dropna()

newest_df_copy

,rank,song,pattern,kr_song,kr_composer
0,1,애프터 라이크,E01,애프터 라이크,아이브
1,2,애프터 라이크,N04,애프터 라이크,아이브
2,3,톰보이,N03,톰보이,(여자)아이들
3,4,애프터 라이크,H06,애프터 라이크,아이브
4,5,톰보이,H05,톰보이,(여자)아이들
...,...,...,...,...,...
3833,3834,미션 파서블,S05,미션 파서블,반야
3834,3835,마이 드림즈,S04,마이 드림즈,반야 프로덕션
3835,3836,알레그로 피유 모쏘,S04,알레그로 피유 모쏘,DM Ashura
3836,3837,아크 오브 다크니스,S02,아크 오브 다크니스,이 얍
